# **League of legends API**
 
Documentation: https://developer.riotgames.com/apis

In [4]:
import requests
from time import sleep
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import datetime
load_dotenv()

True

In [5]:
#api_key = os.getenv("RIOT_API_KEY")
api_key = 'RGAPI-3bba9d89-2b02-4131-a72d-8ec8e0c17700'

## **Account**

In [6]:
summoner_name = 'Styles'

endpoint = f'https://la1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={api_key}'
summoner = requests.get(endpoint).json()

In [7]:
df = pd.DataFrame([summoner])
df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,5QDTqVFS66ywwKzRVgBxnrZLFL3IbIMhndHAYWaGIgA3eEE,IosCQt_vyGYT5QGNSQasm-pP24wQPww9xv96DiXkkyOAdm4,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,Styles,5922,1712025815000,691


## **Ranking Mastery**

In [8]:
summ_id = summoner['id']
endpoint = f'https://la1.api.riotgames.com/lol/league/v4/entries/by-summoner/{summ_id}?api_key={api_key}'

res = requests.get(endpoint).json()
df_rank = pd.DataFrame([res[0]])
df_rank

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,bb9feb00-cf13-40ca-b318-0a74573845b5,RANKED_SOLO_5x5,DIAMOND,IV,5QDTqVFS66ywwKzRVgBxnrZLFL3IbIMhndHAYWaGIgA3eEE,Styles,48,61,45,False,False,False,False


In [9]:
df_summ = pd.concat([df, df_rank] ,axis=1)

Here we have duplicated or unnecesary rows 'summonerId','SummonerName', therefore we drop them

In [10]:
df_summ.dtypes

id               object
accountId        object
puuid            object
name             object
profileIconId     int64
revisionDate      int64
summonerLevel     int64
leagueId         object
queueType        object
tier             object
rank             object
summonerId       object
summonerName     object
leaguePoints      int64
wins              int64
losses            int64
veteran            bool
inactive           bool
freshBlood         bool
hotStreak          bool
dtype: object

In [11]:
df_summ = df_summ.drop(columns=['summonerId','summonerName'])

In [12]:
df_summ

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,leagueId,queueType,tier,rank,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,5QDTqVFS66ywwKzRVgBxnrZLFL3IbIMhndHAYWaGIgA3eEE,IosCQt_vyGYT5QGNSQasm-pP24wQPww9xv96DiXkkyOAdm4,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,Styles,5922,1712025815000,691,bb9feb00-cf13-40ca-b318-0a74573845b5,RANKED_SOLO_5x5,DIAMOND,IV,48,61,45,False,False,False,False


In [13]:
# Lets transform revisionDate from epoch to datetime
df_summ['revisionDate']

0    1712025815000
Name: revisionDate, dtype: int64

In [14]:
df_summ['revisionDate'] = pd.to_datetime(df['revisionDate'],unit='ms')
df_summ['revisionDate']

0   2024-04-02 02:43:35
Name: revisionDate, dtype: datetime64[ns]

Save the data into a file

In [15]:
df_summ.to_csv(f'data/summoner_{datetime.today().strftime("%Y_%m_%d")}.csv', index=False)

## **Champion mastery**

In [16]:
server = 'la1'
summ_puuid = summoner['puuid']
endpoint = f'https://{server}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{summ_puuid}/top?api_key={api_key}'

res = requests.get(endpoint).json()

In [17]:
df_champs = pd.DataFrame(res)
df_champs

,puuid,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,245,7,404598,1694360659000,382998,0,False,0,z3o6lqYCYmy0fK8C7SR5E93YkDuU2KcSjrq_ihbpuE56Y8pE
1,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,55,7,278059,1712025816000,256459,0,True,0,z3o6lqYCYmy0fK8C7SR5E93YkDuU2KcSjrq_ihbpuE56Y8pE
2,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,141,7,259593,1705030903000,237993,0,False,0,z3o6lqYCYmy0fK8C7SR5E93YkDuU2KcSjrq_ihbpuE56Y8pE


In [18]:
df_champs['lastPlayTime'] = pd.to_datetime(df_champs['lastPlayTime'],unit='ms')
df_champs.to_csv(f'data/champs_mastery_{datetime.today().strftime("%Y_%m_%d")}.csv', index=False)

## **Match history**

In [36]:
summ_puuid = summoner['puuid']
match_server='americas.api.riotgames.com'
endpoint = f'https://{match_server}/lol/match/v5/matches/by-puuid/{summ_puuid}/ids?api_key={api_key}'
res = requests.get(endpoint).json()
res

['LA1_1503832749',
 'LA1_1503820745',
 'LA1_1503814083',
 'LA1_1502725218',
 'LA1_1502714802',
 'LA1_1501610807',
 'LA1_1501602761',
 'LA1_1501562459',
 'LA1_1500786367',
 'LA1_1500772843',
 'LA1_1500758832',
 'LA1_1499538538',
 'LA1_1499396930',
 'LA1_1499388826',
 'LA1_1499360152',
 'LA1_1499352190',
 'LA1_1499345383',
 'LA1_1498966218',
 'LA1_1498917144',
 'LA1_1498907137']

In [37]:
match_id = res[7]
endpoint = f'https://{match_server}/lol/match/v5/matches/{match_id}?api_key={api_key}'

res = requests.get(endpoint).json()

In [31]:
basics = [
    'puuid',
    'riotIdGameName',
    'win',
    'lane',
    'role',
    'teamPosition',
    'individualPosition',
    'kills',
    'deaths',
    'assists',
    'totalMinionsKilled',
    'eligibleForProgression',
    'timePlayed',
    'gameEndedInEarlySurrender',
    'gameEndedInSurrender',
]

kills = [
    'killingSprees',
    'firstBloodAssist',
    'firstBloodKill',
    'firstTowerAssist',
    'firstTowerKill',
    'doubleKills',
    'tripleKills',
    'quadraKills',
    'pentaKills',
]

wards = [
    'wardsKilled',
    'wardsPlaced',
    'visionWardsBoughtInGame',
    'detectorWardsPlaced',
]

objectives = [
    'objectivesStolen',
    'turretKills',
    'dragonKills',
    'inhibitorKills',
    'nexusKills',
]

pings = [
    'dangerPings',
    'enemyMissingPings',
    'enemyVisionPings',
    'getBackPings',
    'holdPings',
    'needVisionPings',
    'onMyWayPings',
    'pushPings',
    'visionClearedPings',
]

misc = [
    'item0',
    'item1',
    'item2',
    'item3',
    'item4',
    'item5',
    'item6',
    'goldEarned',
    'goldSpent',
    'inhibitorTakedowns',
    'inhibitorsLost',
    'itemsPurchased',
    'largestCriticalStrike',
    'largestKillingSpree',
    'largestMultiKill',
    'longestTimeSpentLiving',
    'magicDamageDealt',
    'magicDamageDealtToChampions',
    'magicDamageTaken',
    'neutralMinionsKilled',
    'nexusLost',
    'nexusTakedowns',
    'objectivesStolenAssists',
    'participantId',
    'physicalDamageDealt',
    'physicalDamageDealtToChampions',
    'physicalDamageTaken',
    'sightWardsBoughtInGame',
    'spell1Casts',
    'spell2Casts',
    'spell3Casts',
    'spell4Casts',
    'summoner1Casts',
    'summoner1Id',
    'summoner2Casts',
    'summoner2Id',
    'teamEarlySurrendered',
    'teamId',
    'timeCCingOthers',
    'totalDamageDealt',
    'totalDamageDealtToChampions',
    'totalDamageShieldedOnTeammates',
    'totalDamageTaken',
    'totalHeal',
    'totalHealsOnTeammates',
    'totalTimeCCDealt',
    'totalTimeSpentDead',
    'totalUnitsHealed',
    'trueDamageDealt',
    'trueDamageDealtToChampions',
    'trueDamageTaken',
    'turretTakedowns',
    'turretsLost',
    'unrealKills',
    'visionScore',
]
stats = basics + kills + wards + objectives + pings + misc


In [32]:
df = pd.DataFrame(res['info']['participants'])
df_match = df[stats]

In [33]:
df_match

,puuid,riotIdGameName,win,lane,role,teamPosition,individualPosition,kills,deaths,assists,...,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretTakedowns,turretsLost,unrealKills,visionScore
0,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,Styles,True,TOP,SOLO,TOP,TOP,13,6,11,...,157,239,1,13792,7083,1246,6,3,0,6
1,N2IMi9hTN5QKB-0Dh__jLCjZtTZkW514axXSolUccRIY4m...,hxngr,True,JUNGLE,NONE,JUNGLE,JUNGLE,8,5,13,...,1077,143,1,55105,739,1221,3,3,0,19
2,ozPLC1rJK1JtRGFOgSzX8ZSFjQpmrGrtGWiHFX3o7w3FL4...,Neoz,True,MIDDLE,SOLO,MIDDLE,MIDDLE,9,8,15,...,95,230,1,36650,6591,357,5,3,0,20
3,aHuJ4DzCRV-eXWi1zbLx7eN3A4hWiGFldPMKZ3N3KCJEzC...,Hide on Sea,True,BOTTOM,CARRY,BOTTOM,BOTTOM,9,9,8,...,203,222,1,14583,252,255,8,3,0,13
4,g8E0e2bUObrsxQIbT5y0vwUaf2ZBWY5kAOYO2PiR_ZhKod...,Täuki,True,BOTTOM,SUPPORT,UTILITY,UTILITY,1,8,22,...,273,209,5,2224,0,814,5,3,0,51
5,dyBNjm7fqDX3I8i6Z1QatWEMrKrZ9kuslRMceuwtCqKiyr...,Chosen One,False,TOP,DUO,TOP,TOP,3,9,5,...,130,212,1,1139,1079,3408,1,11,0,18
6,YvOPUmCtrEKWzv7r7tDBd3g2iU7eiSmJ5g6ULE0hBScZld...,WadeVanrockels,False,JUNGLE,NONE,JUNGLE,JUNGLE,3,11,19,...,650,232,1,63142,2767,1066,0,11,0,13
7,ftgxUyFDFmnj_otWUEu-Fa9V7j0Q8NVwde6xphPGtBIREl...,Jk1n,False,TOP,DUO,MIDDLE,MIDDLE,13,7,9,...,389,269,1,3271,0,7358,1,11,0,18
8,tILegJASy62obGyU0oXsLWkKfDtjlkfcp1GbFbpqArbkME...,Spxrk,False,BOTTOM,CARRY,BOTTOM,BOTTOM,11,6,8,...,202,138,1,254,48,1285,1,11,0,25
9,KpROXtBe14dYprHhnSrWRMpqRf5KYCUCNMl3-cOMElr4FX...,Mc Lovin 0001,False,BOTTOM,SUPPORT,UTILITY,UTILITY,6,7,19,...,275,152,5,1489,0,1547,1,11,0,70


Export the match information (specifically the last one)

In [34]:
df_match.to_csv(f'data/match_info{datetime.today().strftime("%Y_%m_%d")}.csv', index=False)

## **Extract info from Items**

In [39]:
version_lst = requests.get('http://ddragon.leagueoflegends.com/api/versions.json').json()
endpoint = f'http://ddragon.leagueoflegends.com/cdn/{version_lst[0]}/data/en_US/item.json'
res = requests.get(endpoint).json()
#res['data']

In [50]:
item_lst = []
for iid, data in res['data'].items():
    item = {
        'id' : iid,
        'name': data['name'],
        'descripcion': data['description'],
        'plaintext': data['plaintext'],
        'stats': data['stats'],
        'gold_base': data['gold']['base'],
        'gold_total': data['gold']['total'],
        'gold_sell': data['gold']['sell'],
    }
    item_lst.append(item)
df_items = pd.DataFrame(item_lst)

In [51]:
df_items.to_csv(f'data/items_info{datetime.today().strftime("%Y_%m_%d")}.csv', index=False)

## **Extract info from Champions**

In [38]:
version_lst = requests.get('http://ddragon.leagueoflegends.com/api/versions.json').json()
endpoint = f'http://ddragon.leagueoflegends.com/cdn/{version_lst[0]}/data/en_US/champion.json'
res = requests.get(endpoint).json()
#res

In [53]:
champ_lst = []
for iid, data in res['data'].items():
    champ = {
        'id' : iid,
        'name': data['name'],
        'title': data['title'],
        'blurb': data['blurb'],
        'tags': data['tags'],
        'partype': data['partype'],
        'stats': data['stats'],
    }
    champ_lst.append(champ)
df_champs = pd.DataFrame(champ_lst)
df_champs

,id,name,title,blurb,tags,partype,stats
0,Aatrox,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"[Fighter, Tank]",Blood Well,"{'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpper..."
1,Ahri,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,"[Mage, Assassin]",Mana,"{'hp': 590, 'hpperlevel': 104, 'mp': 418, 'mpp..."
2,Akali,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,[Assassin],Energy,"{'hp': 570, 'hpperlevel': 119, 'mp': 200, 'mpp..."
3,Akshan,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...","[Marksman, Assassin]",Mana,"{'hp': 630, 'hpperlevel': 107, 'mp': 350, 'mpp..."
4,Alistar,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"[Tank, Support]",Mana,"{'hp': 685, 'hpperlevel': 120, 'mp': 350, 'mpp..."
...,...,...,...,...,...,...,...
162,Zeri,Zeri,The Spark of Zaun,"A headstrong, spirited young woman from Zaun's...",[Marksman],Mana,"{'hp': 630, 'hpperlevel': 110, 'mp': 250, 'mpp..."
163,Ziggs,Ziggs,the Hexplosives Expert,"With a love of big bombs and short fuses, the ...",[Mage],Mana,"{'hp': 606, 'hpperlevel': 106, 'mp': 480, 'mpp..."
164,Zilean,Zilean,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...","[Support, Mage]",Mana,"{'hp': 574, 'hpperlevel': 96, 'mp': 452, 'mppe..."
165,Zoe,Zoe,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...","[Mage, Support]",Mana,"{'hp': 630, 'hpperlevel': 106, 'mp': 425, 'mpp..."


In [54]:
df_stats = pd.json_normalize(df_champs['stats'])
df_stats

,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,spellblockperlevel,attackrange,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,650,114,0,0.0,345,38,4.45,32,2.05,175,3.00,1.00,0.00,0.00,0,0,60,5.00,2.500,0.651
1,590,104,418,25.0,330,21,4.70,30,1.30,550,2.50,0.60,8.00,0.80,0,0,53,3.00,2.200,0.668
2,570,119,200,0.0,345,23,4.70,37,2.05,125,9.00,0.90,50.00,0.00,0,0,62,3.30,3.200,0.625
3,630,107,350,40.0,330,26,4.70,30,1.30,500,3.75,0.65,8.20,0.70,0,0,52,3.00,4.000,0.638
4,685,120,350,40.0,330,47,4.70,32,2.05,125,8.50,0.85,8.50,0.80,0,0,62,3.75,2.125,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,630,110,250,45.0,330,24,4.20,30,1.30,500,3.25,0.70,6.00,0.80,0,0,56,2.00,2.000,0.658
163,606,106,480,23.5,325,21,4.70,30,1.30,550,6.50,0.60,8.00,0.80,0,0,55,3.10,2.000,0.656
164,574,96,452,50.0,335,24,5.00,30,1.30,550,5.50,0.50,11.35,0.80,0,0,52,3.00,2.130,0.658
165,630,106,425,25.0,340,21,4.70,30,1.30,550,7.50,0.60,8.00,0.65,0,0,58,3.30,2.500,0.658


In [55]:
df_champ_stats = pd.concat([df_champs, df_stats],axis=1)
df_champ_stats = df_champ_stats.drop(columns=['stats'])
df_champ_stats.to_csv(f'data/champions_info{datetime.today().strftime("%Y_%m_%d")}.csv', index=False)